In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import LeakyReLU
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image as keras_image
import face_recognition

In [ ]:
# Path

train = ""

In [ ]:
#read dataset for training
def load_data_from_folder(folder_path):
    images = []
    labels = []

    for name in os.listdir(folder_path):
        path = os.path.join(folder_path, name)
        if os.path.isdir(path):
            for filename in os.listdir(path):
                if filename.endswith('.jpg'):
                    image = cv2.imread(os.path.join(path, filename))
                    images.append(image)
                    labels.append(name)

    return images, labels

In [ ]:
data_x, data_y = load_data_from_folder(train)

In [ ]:
data_x = np.array(data_x)
data_y = np.array(data_y)

In [ ]:
label_encoder = LabelEncoder()
data_y = label_encoder.fit_transform(data_y)

In [ ]:
np.unique(data_y)

In [ ]:
data_x = data_x / 255.0

In [ ]:
data_x

# Inception

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2)

In [ ]:
inception_model = models.Sequential([
    layers.InputLayer((48, 48, 3)),
    
    layers.Conv2D(64, (7, 7), strides=(2, 2)),
    LeakyReLU(alpha=0.1),
    layers.MaxPooling2D((3, 3), strides=(2, 2)),
    
    layers.Conv2D(192, (3, 3), strides=(1, 1)),
    LeakyReLU(alpha=0.1),
    layers.MaxPooling2D((3, 3), strides=(2, 2)),
    
    # inception layer 1
    layers.Conv2D(64, (1, 1), padding='same'),
    LeakyReLU(alpha=0.1),
    layers.Conv2D(128, (3, 3), padding='same'),
    LeakyReLU(alpha=0.1),
    layers.Conv2D(96, (1, 1), padding='same'),
    LeakyReLU(alpha=0.1),
    layers.Conv2D(32, (5, 5), padding='same'),
    LeakyReLU(alpha=0.1),
    layers.Conv2D(16, (1, 1), padding='same'),
    LeakyReLU(alpha=0.1),
    
    layers.MaxPooling2D((2, 2), strides=(2, 2)),
    
    layers.GlobalAveragePooling2D(),
    layers.Dense(4096),
    LeakyReLU(alpha=0.1),
    layers.Dense(1024),
    LeakyReLU(alpha=0.1),
    
    layers.Dense(4, activation='softmax')
])

In [ ]:
inception_model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

inception_model.fit(x_train, y_train, epochs=60, batch_size=100, validation_data=(x_test, y_test))

In [ ]:
test_loss, test_acc = inception_model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

In [ ]:
#load NAO pictures from file for testing
def load_data_from_folder_test(folder_path):
    images = []
    labels = []

    for name in os.listdir(folder_path):
        path = os.path.join(folder_path, name)
        if os.path.isdir(path):
            for filename in os.listdir(path):
                if filename.endswith('.jpeg'):
                    
                    image_path = os.path.join(path, filename) 
                    
                    image = cv2.imread(image_path, cv2.COLOR_RGBA2BGR)
                                                                            
                    cropped_faces = crop_faces(image)
                                        
                    #replicated_image = np.stack((cropped_faces,) * 3, axis=-1)
                    for face in cropped_faces:
                        
                    
                        normalized_image = face / 255.0
                    
                        images.append(normalized_image)
                        labels.append(name)
                                        
                    for i, face in enumerate(cropped_faces):
                        plt.imshow(face)
                        plt.show()

    return images, labels

def crop_faces(image):
    face_locations = face_recognition.face_locations(image)   
    cropped_faces = []
    for face_location in face_locations:
        top, right, bottom, left = face_location     
        cropped_face = image[top:bottom, left:right]
        cropped_face = cv2.cvtColor(cropped_face, cv2.COLOR_RGBA2BGR)
        cropped_face = cv2.resize(cropped_face, (48, 48))
        cropped_faces.append(cropped_face)
    return cropped_faces

In [ ]:
test_folder = ""
nao_x, nao_y = load_data_from_folder_test(test_folder)

nao_x = np.array(nao_x)
nao_y = np.array(nao_y)

In [ ]:
label_encoder = LabelEncoder()
nao_y = label_encoder.fit_transform(nao_y)

In [ ]:
test_loss, test_acc = inception_model.evaluate(nao_x, nao_y)
print(f'Test accuracy: {test_acc}')

# CNN

In [ ]:
cnn_model = models.Sequential([
    layers.InputLayer((48,48,3),100),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(4096, activation='relu'),
    layers.Dense(4096, activation='relu'),
    layers.Dense(4, activation='softmax')
])


In [ ]:
cnn_model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

cnn_model.fit(x_train, y_train, epochs=40, batch_size=100, validation_data=(x_test, y_test))


test_loss, test_acc = cnn_model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

# VGG

In [ ]:
vgg16_layer = models.Sequential([
    layers.InputLayer((48,48,3),32),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(512, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(1024, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.GlobalAveragePooling2D(),
    layers.Dense(4, activation='softmax')
])

In [ ]:
vgg16_layer.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

vgg16_layer.fit(train_x, train_y, epochs=50, batch_size=32, validation_data=(test_x, test_y))


test_loss, test_acc = vgg16_layer.evaluate(test_x, test_y)
print(f'Test accuracy: {test_acc}')